In [ ]:
try:
  # %tensorflow_version  only  exists  in  Colab.
  %tensorflow_version  2.x
except  Exception:
  pass

TensorFlow 2.x selected.


In [ ]:
# Linear Algebra
import numpy as np
# Data processing
import pandas as pd
# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from  tensorflow  import  keras
from  keras.optimizers  import  SGD
from  keras.utils  import  to_categorical
import sklearn

Using TensorFlow backend.


In [ ]:
tf.__version__

'2.1.0'

In [ ]:
movies = pd.read_csv("/content/sample_data/movies.csv")
ratings = pd.read_csv("/content/sample_data/ratings.csv")

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
def extractor(line):
    """
    this function receives a row from the movies dataframe and extracts its attributes.
    """

    parantheses_open = [pos for pos, char in enumerate(line) if char == '(']
    parantheses_close = [pos for pos, char in enumerate(line) if char == ')']

    if parantheses_open == []:
        "if the movie year is not given, we define it equal to 0"
        return (line ,0)

    movie = line[0 : parantheses_open[-1]-1]
    year = line[parantheses_open[-1] + 1 : parantheses_close[-1]]

    if '-' in year:
        'inside ////'
        years = year.split('-')
        year = [int(item) for item in years]

    else:
        year = int(year)

    return (movie, year)

In [ ]:
movies["year"] = 0


In [ ]:
all_genres = set()
for index, row in movies.iterrows():
    #print(extractor(row['title']))
    genres_list = row['genres'].split('|')
    for gen in genres_list:
        all_genres.add(gen)

In [ ]:
all_genres = list(all_genres)
len(all_genres)

19

In [ ]:
print(all_genres)
all_genres.index('Drama')


['Western', 'Adventure', 'Fantasy', 'Romance', 'Animation', 'Sci-Fi', 'Crime', 'Thriller', 'Horror', 'Action', 'War', 'Children', 'Mystery', 'Drama', 'IMAX', 'Film-Noir', 'Documentary', 'Musical', 'Comedy']


13

In [ ]:
movies_new_columns = ['movieId', 'title', 'year']+all_genres
print(len(movies_new_columns))
print(len(all_genres))
movies_new = pd.DataFrame(columns = movies_new_columns)
print(movies_new)

22
19
Empty DataFrame
Columns: [movieId, title, year, Western, Adventure, Fantasy, Romance, Animation, Sci-Fi, Crime, Thriller, Horror, Action, War, Children, Mystery, Drama, IMAX, Film-Noir, Documentary, Musical, Comedy]
Index: []


In [ ]:
for index, row in movies.iterrows():
  (title, year) = extractor(row['title'])
  
  genres_vector = [0.0]*len(all_genres)
  genres_list = row['genres'].split('|')
  for g in genres_list:
    genres_vector[all_genres.index(g)] = 1.0

  list_to_append = [row['movieId'], title, year] + genres_vector
  dic = {movies_new_columns[i]: list_to_append[i] for i in range(len(list_to_append))} 
  movies_new = movies_new.append(dic, ignore_index=True)

In [ ]:
movies_new

,movieId,title,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,Toy Story,1995,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,Jumanji,1995,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,Waiting to Exhale,1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5,Father of the Bride Part II,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,4988,White Water Summer,1987,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2739,4989,How High,2001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2740,4990,Jimmy Neutron: Boy Genius,2001,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2741,4991,Joe Somebody,2001,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
ratings['movieId']=ratings['movieId'].astype(int)
movies_new['movieId']= movies_new['movieId'].astype(int)

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
genres_df = movies_new.iloc[:,3:]
genres_df

,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2740,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2741,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
genres_df_normalised = genres_df.div(genres_df.sum(axis=1), axis=0)
genres_df_normalised

,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,0.0,0.200000,0.200000,0.000000,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.200000
1,0.0,0.333333,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.000000,0.500000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.500000
3,0.0,0.000000,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333
4,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,0.0,1.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2739,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000
2740,0.0,0.250000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000
2741,0.0,0.000000,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333


In [ ]:
movies_new = pd.concat([movies_new.iloc[:,0:3], genres_df_normalised], axis=1, sort = False)
movies_new

,movieId,title,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,Toy Story,1995,0.0,0.200000,0.200000,0.000000,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.200000
1,2,Jumanji,1995,0.0,0.333333,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,3,Grumpier Old Men,1995,0.0,0.000000,0.000000,0.500000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.500000
3,4,Waiting to Exhale,1995,0.0,0.000000,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333
4,5,Father of the Bride Part II,1995,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,4988,White Water Summer,1987,0.0,1.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2739,4989,How High,2001,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000
2740,4990,Jimmy Neutron: Boy Genius,2001,0.0,0.250000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000
2741,4991,Joe Somebody,2001,0.0,0.000000,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333


In [ ]:
 result = pd.merge(ratings,
                     movies_new[movies_new_columns],
                     on='movieId')
result.head()

,userId,movieId,rating,timestamp,title,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,1,4.0,964982703,Toy Story,1995,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2
1,5,1,4.0,847434962,Toy Story,1995,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2
2,7,1,4.5,1106635946,Toy Story,1995,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2
3,15,1,2.5,1510577970,Toy Story,1995,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2
4,17,1,4.5,1305696483,Toy Story,1995,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2


In [ ]:
result = result.drop(['timestamp', 'title'], axis=1)

In [ ]:
result

,userId,movieId,rating,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,1,4.0,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
1,5,1,4.0,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
2,7,1,4.5,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
3,15,1,2.5,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
4,17,1,4.5,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50040,607,2737,4.0,1987,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.333333,0.00,0.333333,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.00
50041,609,137,3.0,1995,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.00
50042,610,1054,2.0,1996,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.00
50043,610,3945,3.0,2000,0.0,0.333333,0.0,0.0,0.333333,0.00,0.0,0.000000,0.00,0.000000,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.00


In [ ]:
# users data frame
users_columns = ['userId']+all_genres
users_new = pd.DataFrame(columns = users_columns)
print(users_new)

Empty DataFrame
Columns: [userId, Western, Adventure, Fantasy, Romance, Animation, Sci-Fi, Crime, Thriller, Horror, Action, War, Children, Mystery, Drama, IMAX, Film-Noir, Documentary, Musical, Comedy]
Index: []


In [ ]:
result.sort_values(['userId', "movieId"], ascending = (True, True))

,userId,movieId,rating,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,1,4.0,1995,0.0,0.20,0.2,0.00,0.20,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.20,0.00,0.000000,0.0,0.0,0.0,0.0,0.20
215,1,3,4.0,1995,0.0,0.00,0.0,0.50,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.50
267,1,6,4.0,1995,0.0,0.00,0.0,0.00,0.00,0.00,0.333333,0.333333,0.00,0.333333,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.00
369,1,47,5.0,1995,0.0,0.00,0.0,0.00,0.00,0.00,0.000000,0.500000,0.00,0.000000,0.000000,0.00,0.50,0.000000,0.0,0.0,0.0,0.0,0.00
572,1,70,3.0,1996,0.0,0.00,0.0,0.00,0.00,0.00,0.000000,0.250000,0.25,0.250000,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39417,610,4901,3.5,2001,0.0,0.00,0.0,0.00,0.00,0.00,0.250000,0.250000,0.00,0.250000,0.000000,0.00,0.00,0.250000,0.0,0.0,0.0,0.0,0.00
43595,610,4958,3.5,2001,0.0,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.00,0.333333,0.333333,0.00,0.00,0.333333,0.0,0.0,0.0,0.0,0.00
27149,610,4963,4.0,2001,0.0,0.00,0.0,0.00,0.00,0.00,0.500000,0.500000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.00
43717,610,4975,3.5,2001,0.0,0.00,0.0,0.25,0.00,0.25,0.000000,0.250000,0.00,0.000000,0.000000,0.00,0.25,0.000000,0.0,0.0,0.0,0.0,0.00


In [ ]:
for user_id in range(1,611,1):
  #print(user_id)
  user = result[result['userId'] == user_id]

  new_line = []
  for g in all_genres:
    tempo = user[["rating"]].multiply(user[g], axis="index")
    new_line.append(tempo.sum(axis = 0, skipna = True)[0])

  list_to_append = [user_id] + new_line
  line_lables = ['userId'] + all_genres
  dic = {line_lables[i]: list_to_append[i] for i in range(len(list_to_append))} 
  users_new = users_new.append(dic, ignore_index=True)

In [ ]:
users_new

,userId,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1.0,10.166667,86.397619,43.730952,26.000000,21.347619,42.483333,46.697619,48.716667,13.666667,95.283333,24.500000,31.897619,18.130952,84.900000,0.000000,1.250000,0.0,19.916667,133.914286
1,2.0,0.000000,1.333333,0.000000,2.250000,0.000000,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,0.000000,0.000000,3.583333,0.000000,0.000000,0.0,0.000000,4.000000
2,3.0,0.000000,8.041667,2.833333,0.666667,0.458333,7.333333,0.125000,2.916667,4.750000,4.833333,0.750000,0.458333,0.000000,2.750000,0.000000,0.000000,0.0,0.125000,3.458333
3,4.0,13.333333,24.145238,16.333333,57.095238,2.978571,12.500000,28.916667,31.250000,2.000000,22.250000,5.750000,5.311905,21.666667,165.511905,0.500000,3.416667,8.0,11.228571,152.811905
4,5.0,1.916667,6.083333,5.883333,9.316667,4.683333,2.166667,12.833333,7.483333,1.500000,8.400000,4.083333,7.683333,0.000000,36.833333,1.833333,0.000000,0.0,3.883333,15.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606.0,9.375000,63.028571,42.670238,253.833333,6.795238,50.366667,71.811905,113.033333,41.000000,71.483333,42.916667,9.045238,48.428571,651.566667,0.000000,4.866667,2.0,25.908333,332.870238
606,607.0,3.333333,38.116667,14.100000,25.383333,4.966667,29.450000,22.433333,51.066667,45.583333,74.683333,6.500000,13.333333,16.333333,94.600000,1.666667,0.000000,0.0,2.816667,56.633333
607,608.0,10.416667,89.386905,48.578571,73.533333,20.170238,106.466667,87.736905,152.875000,61.358333,138.700000,15.033333,35.486905,34.053571,243.033333,1.083333,2.333333,0.0,9.866667,301.386905
608,609.0,1.333333,9.516667,0.600000,6.500000,0.600000,5.000000,5.000000,12.333333,3.500000,10.333333,3.000000,0.600000,0.000000,21.083333,1.000000,0.000000,6.0,0.000000,9.600000


In [ ]:
movies_new

,movieId,title,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,Toy Story,1995,0.0,0.200000,0.200000,0.000000,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.200000
1,2,Jumanji,1995,0.0,0.333333,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,3,Grumpier Old Men,1995,0.0,0.000000,0.000000,0.500000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.500000
3,4,Waiting to Exhale,1995,0.0,0.000000,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333
4,5,Father of the Bride Part II,1995,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,4988,White Water Summer,1987,0.0,1.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2739,4989,How High,2001,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000
2740,4990,Jimmy Neutron: Boy Genius,2001,0.0,0.250000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000
2741,4991,Joe Somebody,2001,0.0,0.000000,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333


In [ ]:
from scipy import spatial

In [ ]:
def Recommandation(userID):
  result_colums = ['movieId', 'title', 'year','cos']
  result = pd.DataFrame(columns = result_colums)
  result = movies_new.iloc[:,:3]
  result['cos'] = 0.0

  user = users_new[users_new["userId"]== userID]
  for i in range(movies_new.shape[0]):
    vectFilm = (movies_new.iloc[i,3:]).to_numpy()
    VectUser = (user.iloc[:,1:]).to_numpy()
    cosinus = 1 - spatial.distance.cosine(VectUser, vectFilm)
    result.at[i,'cos']=cosinus
  return result

In [ ]:
print("Top 3 recommended films for the User 1 :")

RecomUser = Recommandation(1)
RecomUser.sort_values(['cos'], ascending = False,inplace=True)
RecomUser.head(3)

Top 3 recommended films for the User 1 :


,movieId,title,year,cos
2663,4818,Extreme Days,2001,0.863953
1644,2890,Three Kings,1999,0.820015
99,145,Bad Boys,1995,0.790141


In [ ]:
result

,userId,movieId,rating,year,Western,Adventure,Fantasy,Romance,Animation,Sci-Fi,Crime,Thriller,Horror,Action,War,Children,Mystery,Drama,IMAX,Film-Noir,Documentary,Musical,Comedy
0,1,1,4.0,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
1,5,1,4.0,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
2,7,1,4.5,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
3,15,1,2.5,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
4,17,1,4.5,1995,0.0,0.200000,0.2,0.0,0.200000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50040,607,2737,4.0,1987,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.333333,0.00,0.333333,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.00
50041,609,137,3.0,1995,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.00
50042,610,1054,2.0,1996,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.00
50043,610,3945,3.0,2000,0.0,0.333333,0.0,0.0,0.333333,0.00,0.0,0.000000,0.00,0.000000,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.00


In [ ]:
def train_test(ID):
  test = result[result['userId']==ID]
  train = result[result['userId']!=ID]
  return[test,train]

In [ ]:
testset,trainset = train_test(3)

In [ ]:
y_train = trainset['rating']
X_train = trainset.drop(['rating','userId','movieId','year'], axis = 1)

y_test = testset['rating']
X_test = testset.drop(['rating','userId','movieId','year'], axis = 1)

In [ ]:
classes = pd.DataFrame({"classes":[0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]})
classlst=["['classes']_0.0","['classes']_0.5","['classes']_1.0","['classes']_1.5","['classes']_2.0","['classes']_2.5","['classes']_3.0","['classes']_3.5","['classes']_4.0","['classes']_4.5","['classes']_5.0"]
y_testOHE=pd.get_dummies(y_test,prefix=['classes'])
y_testOHE = y_testOHE.reindex(columns=classlst, fill_value= 0)
y_trainOHE=pd.get_dummies(y_train,prefix=['classes'])
y_trainOHE = y_trainOHE.reindex(columns=classlst, fill_value= 0)
y_trainOHE.head()

,['classes']_0.0,['classes']_0.5,['classes']_1.0,['classes']_1.5,['classes']_2.0,['classes']_2.5,['classes']_3.0,['classes']_3.5,['classes']_4.0,['classes']_4.5,['classes']_5.0
0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
X_test=X_test.astype(np.float,copy=True)
X_train=X_train.astype(np.float,copy=True)

In [ ]:
model = keras.Sequential ([
                           keras.layers.Dense(64,input_shape=X_train.shape[1:]),
                           keras.layers.Dense (64,  activation='relu'),
                           keras.layers.Dense (64,  activation='relu'),
                           keras.layers.Dense(11,  activation='softmax')])

In [ ]:
lrate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate= lrate),loss="categorical_crossentropy",metrics =['accuracy'])

In [ ]:
history = model.fit(X_train , y_trainOHE , validation_split=0.2,epochs =20, verbose =1)

Train on 40018 samples, validate on 10005 samples
Epoch 1/20
40018/40018 [==============================] - 3s 85us/sample - loss: 1.8728 - accuracy: 0.2975 - val_loss: 2.0137 - val_accuracy: 0.2691
Epoch 2/20
40018/40018 [==============================] - 3s 84us/sample - loss: 1.8710 - accuracy: 0.2989 - val_loss: 2.0149 - val_accuracy: 0.2677
Epoch 3/20
40018/40018 [==============================] - 3s 76us/sample - loss: 1.8704 - accuracy: 0.2987 - val_loss: 2.0094 - val_accuracy: 0.2733
Epoch 4/20
40018/40018 [==============================] - 3s 75us/sample - loss: 1.8700 - accuracy: 0.2993 - val_loss: 2.0145 - val_accuracy: 0.2657
Epoch 5/20
40018/40018 [==============================] - 3s 79us/sample - loss: 1.8696 - accuracy: 0.2998 - val_loss: 2.0132 - val_accuracy: 0.2671
Epoch 6/20
40018/40018 [==============================] - 3s 77us/sample - loss: 1.8694 - accuracy: 0.2995 - val_loss: 2.0138 - val_accuracy: 0.2698
Epoch 7/20
40018/40018 [==============================] 

In [ ]:
test_loss, test_acc= model.evaluate(X_test,y_testOHE, verbose = 1)

22/22 [==============================] - 0s 209us/sample - loss: 4.0736 - accuracy: 0.0455
